In [31]:
from importlib import reload
import pdfquery

import incident
reload(incident)
from incident import Incident
import timing
reload(timing)
import utils
reload(utils)

<module 'utils' from '/Users/neel/Git/harvardcrime/utils.py'>

In [32]:
def get_text_from_curve(ltcurve):
    # text can be within LTTextBoxHorizontal or LTTextLineHorizontal
    # the problem is that these are interleaved
    # so we select EITHER as they come
    # if we chose the Boxes first and the Lines second then merged those lists,
    # the resulting list would be out of order!
    # e.g. if the true order is B1 L1 B2 B3 L2, the approach we are using
    # gives you the right order... but choosing Boxes and Lines separately
    # gives you B1 B2 B3 L1 L2!!!!
    textual_elements = ltcurve.cssselect("LTTextBoxHorizontal, LTTextLineHorizontal")
    texts = [t.text.strip() for t in textual_elements]
    
    # remove empty lines
    cleaned_texts = [t for t in texts if t != '']
    
    # PROBLEM with this approach: in rare cases some text from this falls way
    # outside the ltcurve. Might it still be within the bounding box though?
    
    # UPDATE: try gathering all data
    # for 11/28 consider these bboxes
    #
    # 10:08am [247.08, 80.197, 275.587, 90.18]
    # hp laptop [6.0, 65.784, 737.868, 82.318]
    # bounding box [3.36, 77.7, 754.86, 98.64]
    
    return cleaned_texts

In [33]:
def incidents_of_pdf(pdf_file):
    """
    Pass this function the result of a pdfquery.PDFQuery() function.
    This will read through the pdf file and return a list of 
    Incident objects contained in there!
    
    Make sure the PDF is load()'ed before you pass it.
    """
    
    # TODO watch out for things like 11/24/17 where there were no incidents. there's a specific tag for those.
    
    # so each individual report, as well as headers, is filed inside 
    # its own <LTCurve>. The text fields are inside <LTTextLineHorizontal>s and <LTTextBoxHorizontal>s
    # inside the <LTCurve>.
    reports_plus_heads = pdf.tree.findall(".//LTCurve")
    
    # extract raw incidents
    raw_incidents = [get_text_from_curve(lt) for lt in reports_plus_heads]

    # remove headers of tables
    HEADER_ROW_TEXT = ['Reported', 'Incident Type', 'Occurred', 'Location', 'Disposition']
    incidents_without_headers = [i for i in raw_incidents if i != HEADER_ROW_TEXT]
    
    # convert incidents to proper objects
    # 9 = proper length of report; anything less is malformed
    # TODO clean up — extract error checking into its own make_incident_objects() function
    incident_objects = [incident.Incident(i) for i in incidents_without_headers if len(i) == 9]
    
    return incident_objects

In [34]:
pdf = pdfquery.PDFQuery("data/112817.pdf")
pdf.load()
all_incidents = incidents_of_pdf(pdf)


In [35]:
# try another date
pdf = pdfquery.PDFQuery("data/113017.pdf")
pdf.load()
new_incidents = incidents_of_pdf(pdf)
all_incidents += new_incidents
utils.dump_csv(all_incidents)

In [36]:
# REMAINING TODOS:
# - have a function to programmatically download HUPD crime logs
# - have another function to run through all downloaded crime logs in the `data` folder
#   (requires us to read the file system?)
# - Extract the descriptive test along with the metadata. This is somewhat harder but still
#   very important!

In [40]:
# Go through EVERY pdf in our data folder!
import glob

all_incidents = []

for filename in glob.iglob('data/*.pdf'):
    # filename will be like `data/xxxxxx.pdf`
    # extract incidents from this file
    pdf = pdfquery.PDFQuery(filename)
    pdf.load()
    new_incidents = incidents_of_pdf(pdf)
    all_incidents += new_incidents
    
    print("Done {}".format(filename))
    
# dump to csv
utils.dump_csv(all_incidents)
print("Dumped!")

Done data/012018.pdf
Done data/010418.pdf
Done data/020518.pdf
Done data/110817.pdf
Done data/020718.pdf
Done data/010618.pdf
Done data/012218.pdf
Done data/112817.pdf
Done data/122917.pdf
Done data/012618.pdf
Done data/123017.pdf
Done data/010218.pdf
Done data/020318.pdf
Done data/020118.pdf
Done data/011918.pdf
Done data/012418.pdf
Done data/020618.pdf
Done data/010718.pdf
Done data/012118.pdf
Done data/010518.pdf
Done data/020418.pdf
Done data/010118.pdf
Done data/012518.pdf
Done data/011818.pdf
Done data/012718.pdf
Done data/122817.pdf
Done data/010318.pdf
Done data/020218.pdf
Done data/113017.pdf
Done data/123117.pdf
Done data/122417.pdf
Done data/121917.pdf
Done data/011618.pdf
Done data/011418.pdf
Done data/012918.pdf
Done data/122617.pdf
Done data/013018.pdf
Done data/010918.pdf
Done data/020818.pdf
Done data/011018.pdf
Done data/021118.pdf
Done data/122217.pdf
Done data/122017.pdf
Done data/021318.pdf
Done data/011218.pdf
Done data/122717.pdf
Done data/012818.pdf
Done data/021

In [89]:
# TESTING STUFF
pdf = pdfquery.PDFQuery("data/021118.pdf")
pdf.load()

In [90]:
pdf.tree.findall(".//LTTextLineHorizontal")

[<Element LTTextLineHorizontal at 0x10f0aba98>,
 <Element LTTextLineHorizontal at 0x10f0aaa98>,
 <Element LTTextLineHorizontal at 0x10f0abe08>,
 <Element LTTextLineHorizontal at 0x10f0a26d8>,
 <Element LTTextLineHorizontal at 0x10ef49278>,
 <Element LTTextLineHorizontal at 0x10ef36b88>,
 <Element LTTextLineHorizontal at 0x10f0a39f8>,
 <Element LTTextLineHorizontal at 0x10ef40598>,
 <Element LTTextLineHorizontal at 0x109dadcc8>,
 <Element LTTextLineHorizontal at 0x10f0a75e8>,
 <Element LTTextLineHorizontal at 0x10f0a8368>,
 <Element LTTextLineHorizontal at 0x10f0a8ef8>,
 <Element LTTextLineHorizontal at 0x109da9638>,
 <Element LTTextLineHorizontal at 0x109dacd18>,
 <Element LTTextLineHorizontal at 0x10ef2ddb8>,
 <Element LTTextLineHorizontal at 0x10f0d55e8>,
 <Element LTTextLineHorizontal at 0x10f0df4f8>,
 <Element LTTextLineHorizontal at 0x10f0f2188>,
 <Element LTTextLineHorizontal at 0x10f0fb098>,
 <Element LTTextLineHorizontal at 0x10f10d9a8>,
 <Element LTTextLineHorizontal at 0x10f1

In [91]:
def comments():
    return this.get('word_margin',0) == "0.1" and this.get('x0', 0) == "6.0" and this.get('x1', 0) == "737.868"

# ="0.1" x0="6.0" x1="737.868')
comment_lines = pdf.pq('LTTextLineHorizontal').filter(comments)

In [94]:
def extract_text(comment_line):
    # sometimes text lives directly in the LTTextLine (which is passed),
    # sometimes it lives within a LTTextBox within the LTTextLine.
    # This standardizes that.
    if comment_line.text != '':
        # Contains text directly
        return comment_line.text
    else:
        text_box = comment_line.find("LTTextBoxHorizontal")
        
        if text_box is not None:
            # This is the Box inside the Line
            return text_box.text
        
        else:
            return None
    
comments = [extract_text(line) for line in comment_lines]
comments

['Officers dispatched to a report of an individual smoking a cigarette on the shuttle bus. Officers arrived and report ',
 'individual gone on arrival. ',
 'Officers dispatched to a report of unwanted guests in the building. Officers arrived and report individuals gone on ',
 'arrival. ',
 'Officers dispatched to a report of an individual yelling in the area. Officers arrived and report area quiet and ',
 'individual gone on arrival. ',
 'Officers dispatched to a report of individuals yelling in the area. Officers arrived and report area quiet and individuals ',
 'gone on arrival. ',
 'Officers dispatched to a report of a noise complaint. Officers arrived and report club quiet. ',
 'Officer took a report over the phone of an individual that was arrested for armed robbery. ',
 'Officer dispatched to take a report of a stolen HUID card, $60.00 in U.S. currency, a recovered stolen wallet valued at ',
 '$30.00, a recovered stolen license, a recovered stolen social security card and recover

In [96]:
# Now smoosh the lines together. Lines beginning with
# "Officer" are probably a new entry. Everything else
# should be lumped with the previous line.

cleaned_comments = []
for comment in comments:
    if comment.startswith("Officer"):
        # proper new comment. add to the list!
        cleaned_comments.append(comment)
    else:
        # it's the next line of the previous comment.
        # add it to the previous comment.
        # these lines usually have spaces between them already
        # so no need to add a new one
        cleaned_comments[-1] += comment
        
cleaned_comments

['Officers dispatched to a report of an individual smoking a cigarette on the shuttle bus. Officers arrived and report individual gone on arrival. ',
 'Officers dispatched to a report of unwanted guests in the building. Officers arrived and report individuals gone on arrival. ',
 'Officers dispatched to a report of an individual yelling in the area. Officers arrived and report area quiet and individual gone on arrival. ',
 'Officers dispatched to a report of individuals yelling in the area. Officers arrived and report area quiet and individuals gone on arrival. ',
 'Officers dispatched to a report of a noise complaint. Officers arrived and report club quiet. ',
 'Officer took a report over the phone of an individual that was arrested for armed robbery. ',
 'Officer dispatched to take a report of a stolen HUID card, $60.00 in U.S. currency, a recovered stolen wallet valued at $30.00, a recovered stolen license, a recovered stolen social security card and recovered stolen credit/debit ca

In [ ]:
# TODO attach these to the incidents - earlier comment lines
# go with earlier incidents - then store in CSV.